In [74]:
import pandas as pd

# История обучения

In [75]:
pd.read_csv('/kaggle/input/diplom-files/History/Diplom-All-History.csv')

,Unnamed: 0,loss,accuracy,val_loss,val_accuracy
0,0,0.420711,0.934929,1.400425,0.721973
1,1,0.009044,1.000000,1.302685,0.744395
2,2,0.004371,1.000000,1.240548,0.757848
3,3,0.002496,1.000000,1.218867,0.760090
4,4,0.001876,1.000000,1.199417,0.762332
5,5,0.001313,1.000000,1.176623,0.766816
6,6,0.000966,1.000000,1.166663,0.764574
7,7,0.000772,1.000000,1.157273,0.766816
8,8,0.000614,1.000000,1.146730,0.769058
9,9,0.000477,1.000000,1.136186,0.769058


# Проверка точности

In [76]:
pip install TensorFlow

Note: you may need to restart the kernel to use updated packages.


In [77]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from keras import layers
import logging
import numpy as np
import PIL.Image as Image

logger = tf.get_logger()
logger.setLevel(logging.ERROR)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,     # масштабирование значений пикселей
    shear_range=0.2,    # сдвиг изображения вдоль оси X
    zoom_range=0.2,     # масштабирование изображения
    rotation_range=10,  # поворот изображения на угол в градусах
    brightness_range=[0.5, 1.5],  # изменение яркости изображения
    horizontal_flip=False, # переворот изображения относительно горизонтальной оси
    vertical_flip=False)  # переворот изображения относительно вертикальной оси  

IMAGE_RES = 224

test_set = train_datagen.flow_from_directory(
    '/kaggle/input/berserk-heroes-validation',   # путь к папке с обучающими изображениями
    target_size=(IMAGE_RES, IMAGE_RES),    # размер изображений
    batch_size=32,             # размер пакета (batch)
    class_mode='categorical')       # тип меток 

Found 441 images belonging to 193 classes.


In [78]:
from math import ceil

STEPS_PER_EPOCH = ceil(len(test_set.filenames) / 128)

# Использование learning rate scheduling
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*10,
  decay_rate=1,
  staircase=False)

fe_link = 'https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5'
feature_extractor = hub.KerasLayer(fe_link, input_shape=(IMAGE_RES, IMAGE_RES, 3))

feature_extractor.trainable = False

model = tf.keras.Sequential([
        feature_extractor,
        layers.Dropout(0.2),
        layers.BatchNormalization(),
        layers.Dense(194, activation='softmax'),
      ])

model.compile(
        optimizer=tf.keras.optimizers.Adam(lr_schedule),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_13 (KerasLayer)  (None, 1280)             2731616   
                                                                 
 dropout_3 (Dropout)         (None, 1280)              0         
                                                                 
 batch_normalization_4 (Batc  (None, 1280)             5120      
 hNormalization)                                                 
                                                                 
 dense_13 (Dense)            (None, 194)               248514    
                                                                 
Total params: 2,985,250
Trainable params: 251,074
Non-trainable params: 2,734,176
_________________________________________________________________


In [79]:
from math import ceil

STEPS_PER_EPOCH = ceil(len(test_set.filenames) / 128)

# Использование learning rate scheduling
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*10,
  decay_rate=1,
  staircase=False)

fe_link = 'https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5'
feature_extractor = hub.KerasLayer(fe_link, input_shape=(IMAGE_RES, IMAGE_RES, 3))

feature_extractor.trainable = False

model = tf.keras.Sequential([
        feature_extractor,
        layers.Dropout(0.2),
        layers.BatchNormalization(),
        layers.Dense(193, activation='softmax'),
      ])

model.compile(
        optimizer=tf.keras.optimizers.Adam(lr_schedule),
        loss='categorical_crossentropy',
        metrics=['accuracy'])

model.load_weights('/kaggle/input/diplom-files/Weights/Diplom-All.h5')

In [80]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_14 (KerasLayer)  (None, 1280)             2731616   
                                                                 
 dropout_4 (Dropout)         (None, 1280)              0         
                                                                 
 batch_normalization_5 (Batc  (None, 1280)             5120      
 hNormalization)                                                 
                                                                 
 dense_14 (Dense)            (None, 193)               247233    
                                                                 
Total params: 2,983,969
Trainable params: 249,793
Non-trainable params: 2,734,176
_________________________________________________________________


In [81]:
class_names = np.array(list(test_set.class_indices.keys()))

test_data = iter(test_set)
image_batch, label_batch = next(test_data)

predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()
predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]

1/1 [==============================] - 1s 948ms/step


In [82]:
predicted_ids = tf.one_hot(predicted_ids, 193)

In [83]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(label_batch, predicted_ids) # вычисляем точность
print(acc) # выводим значение точности

0.78125


In [84]:
print(f'Accuracy on the test set: {100*acc:.2f}%')

Accuracy on the test set: 78.12%


In [85]:
results = []
for i in range(1000):
    test_set = train_datagen.flow_from_directory(
        '/kaggle/input/berserk-heroes-validation',   # путь к папке с обучающими изображениями
        target_size=(IMAGE_RES, IMAGE_RES),    # размер изображений
        batch_size=32,             # размер пакета (batch)
        class_mode='categorical',  # тип меток
        shuffle=True)       
    
    test_data = iter(test_set)
    image_batch, label_batch = next(test_data)

    predicted_batch = model.predict(image_batch)
    predicted_batch = tf.squeeze(predicted_batch).numpy()
    predicted_ids = np.argmax(predicted_batch, axis=-1)
    predicted_ids = tf.one_hot(predicted_ids, 193)
    acc = accuracy_score(label_batch, predicted_ids)
    results.append(acc*100)

print(f'Accuracy on the test set: {sum(results) / len(results)}%')

Found 441 images belonging to 193 classes.
1/1 [==============================] - 0s 421ms/step
Found 441 images belonging to 193 classes.
1/1 [==============================] - 0s 357ms/step
Found 441 images belonging to 193 classes.
1/1 [==============================] - 0s 375ms/step
Found 441 images belonging to 193 classes.
1/1 [==============================] - 0s 424ms/step
Found 441 images belonging to 193 classes.
1/1 [==============================] - 0s 380ms/step
Found 441 images belonging to 193 classes.
1/1 [==============================] - 0s 352ms/step
Found 441 images belonging to 193 classes.
1/1 [==============================] - 0s 323ms/step
Found 441 images belonging to 193 classes.
1/1 [==============================] - 0s 328ms/step
Found 441 images belonging to 193 classes.
1/1 [==============================] - 0s 361ms/step
Found 441 images belonging to 193 classes.
1/1 [==============================] - 0s 381ms/step
Found 441 images belonging to 193 classe